# 🤗DeBERTa WeightedLayer + MeanPool KFold with TensorFlow

Hi, everyone! This notebook is a DeBERTaV3 finetuning solution to [Feedback Prize - English Language Learning competition](https://www.kaggle.com/competitions/feedback-prize-english-language-learning). It covers:

* MultilabelStratifiedKFold split of the data
* HuggingFace DeBERTaV3 pre-trained model finetuning with Tensorflow
* WeightedLayerPool + MeanPool TensorFlow implementation
* An approximation of layer-wise learning rate decay
* Ensemble of 5 folds inference
* Submission

In this compitition, I also have the following notebooks:

* LB 0.44 - [RoBERTa finetuning](https://www.kaggle.com/code/electro/fp3-roberta-meanpool-kfold-tensorflow)
* LB 0.46 - [BERT finetuning](https://www.kaggle.com/code/electro/fp3-bert-fine-tuning-tensorflow) | [BERT inference](https://www.kaggle.com/code/electro/fp3-bert-inference-tensorflow)
* LB 0.53 - [basic EDA and bag-of-words solution](https://www.kaggle.com/code/electro/fp3-bag-of-words-tensorflow-starter). 

Please check them out if you are interested. 

If you like this notebook, please upvote it. Thank you.

# Imports

In [1]:
import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
print(f'TF version: {tf.__version__}')
import tensorflow_addons as tfa
from tensorflow.keras import layers

import transformers
print(f'transformers version: {transformers.__version__}')
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

TF version: 2.6.4
transformers version: 4.20.1


In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
#     os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(42)

# Load DataFrame

In [3]:
df = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
display(df.head())
print('\n---------DataFrame Summary---------')
df.info()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5



---------DataFrame Summary---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3911 entries, 0 to 3910
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   text_id      3911 non-null   object 
 1   full_text    3911 non-null   object 
 2   cohesion     3911 non-null   float64
 3   syntax       3911 non-null   float64
 4   vocabulary   3911 non-null   float64
 5   phraseology  3911 non-null   float64
 6   grammar      3911 non-null   float64
 7   conventions  3911 non-null   float64
dtypes: float64(6), object(2)
memory usage: 244.6+ KB


# CV Split

In [4]:
N_FOLD = 5
TARGET_COLS = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

skf = MultilabelStratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(skf.split(df, df[TARGET_COLS])):
    df.loc[val_index, 'fold'] = int(n)
df['fold'] = df['fold'].astype(int)
df['fold'].value_counts()

1    783
0    782
4    782
3    782
2    782
Name: fold, dtype: int64

In [5]:
df.to_csv('./df_folds.csv', index=False)

# Model Config

In [6]:
MAX_LENGTH = 512
BATCH_SIZE = 4
DEBERTA_MODEL = "../input/debertav3base"

Why we should disable dropout in regression task, check this [discussion](https://www.kaggle.com/competitions/commonlitreadabilityprize/discussion/260729).

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(DEBERTA_MODEL)
tokenizer.save_pretrained('./tokenizer/')

cfg = transformers.AutoConfig.from_pretrained(DEBERTA_MODEL, output_hidden_states=True)
cfg.hidden_dropout_prob = 0
cfg.attention_probs_dropout_prob = 0
cfg.save_pretrained('./tokenizer/')

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


# Data Process Function

To make use of HugggingFace DeBERTa model, we have to tokenize our input texts as the pretrained DeBERTa model requires.

In [8]:
def deberta_encode(texts, tokenizer=tokenizer):
    input_ids = []
    attention_mask = []
    
    for text in texts.tolist():
        token = tokenizer(text, 
                          add_special_tokens=True, 
                          max_length=MAX_LENGTH, 
                          return_attention_mask=True, 
                          return_tensors="np", 
                          truncation=True, 
                          padding='max_length')
        input_ids.append(token['input_ids'][0])
        attention_mask.append(token['attention_mask'][0])
    
    return np.array(input_ids, dtype="int32"), np.array(attention_mask, dtype="int32")

In [9]:
def get_dataset(df):
    inputs = deberta_encode(df['full_text'])
    targets = np.array(df[TARGET_COLS], dtype="float32")
    return inputs, targets

# Model

## MeanPool

Instead of using '[CLS]' token, MeanPool method averaging one layer of hidden states along the sequence axis with masking out padding tokens.

## WeightedLayerPool

WeightedLayerPool uses a set of trainable weights to average a set of hidden states from transformer backbone. I use a Dense layer with constraint to implement it.

In [10]:
class MeanPool(tf.keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        embedding_sum = tf.reduce_sum(inputs * broadcast_mask, axis=1)
        mask_sum = tf.reduce_sum(broadcast_mask, axis=1)
        mask_sum = tf.math.maximum(mask_sum, tf.constant([1e-9]))
        return embedding_sum / mask_sum

WeightedLayerPool weights constraints: softmax to push sum(w) to be 1.

In [11]:
class WeightsSumOne(tf.keras.constraints.Constraint):
    def __call__(self, w):
        return tf.nn.softmax(w, axis=0)

## Model Design Choice

Although there are many ways to get your final representations, I choose to take the last 4 layers hidden states of DeBERTa, take MeanPool of them to gather information along the sequence axis, then take WeightedLayerPool with a set of trainable weights to gather information along the depth axis of the model, then finally a regression head.

In [12]:
def get_model():
    input_ids = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), dtype=tf.int32, name="input_ids"
    )
    
    attention_masks = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), dtype=tf.int32, name="attention_masks"
    )
   
    deberta_model = transformers.TFAutoModel.from_pretrained(DEBERTA_MODEL, config=cfg)
    deberta_output = deberta_model.deberta(
        input_ids, attention_mask=attention_masks
    )
    hidden_states = deberta_output.hidden_states
    
    stack_meanpool = tf.stack(
        [MeanPool()(hidden_s, mask=attention_masks) for hidden_s in hidden_states[-4:]], 
        axis=2)
    
    weighted_layer_pool = layers.Dense(1,
                                       use_bias=False,
                                       kernel_constraint=WeightsSumOne())(stack_meanpool)
    
    weighted_layer_pool = tf.squeeze(weighted_layer_pool, axis=-1)
    
    x = layers.Dense(6, activation='linear')(weighted_layer_pool)
    #output = layers.Rescaling(scale=4.0, offset=1.0)(x)
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=x)
    return model

In [13]:
tf.keras.backend.clear_session()
model = get_model()
model.summary()

2022-10-31 21:30:44.047461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 21:30:44.048542: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 21:30:44.049225: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 21:30:44.050086: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 512)]        0                                            
__________________________________________________________________________________________________
deberta (TFDebertaV2MainLayer)  TFBaseModelOutput(la 183831552   input_ids[0][0]                  
                                                                 attention_masks[0][0]            
__________________________________________________________________________________________________
mean_pool (MeanPool)            (None, 768)          0           deberta[0][9]                

# 5 Folds Training Loop

When training model, I employ an approximation of layer-wise learning rate decay. That is a MultiOptimizer with initial learning rate 1e-5 for DeBERTa backbone and initial learning rate 1e-4 for the rest of the model.

In [14]:
valid_rmses = []
for fold in range(N_FOLD):
    print(f'\n-----------FOLD {fold} ------------')
    
    #Create dataset
    train_df = df[df['fold'] != fold].reset_index(drop=True)
    valid_df = df[df['fold'] == fold].reset_index(drop=True)
    train_dataset = get_dataset(train_df)
    valid_dataset = get_dataset(valid_df)
    
    print('Data prepared.')
    print(f'Training data input_ids shape: {train_dataset[0][0].shape} dtype: {train_dataset[0][0].dtype}') 
    print(f'Training data attention_mask shape: {train_dataset[0][1].shape} dtype: {train_dataset[0][1].dtype}')
    print(f'Training data targets shape: {train_dataset[1].shape} dtype: {train_dataset[1].dtype}')
    print(f'Validation data input_ids shape: {valid_dataset[0][0].shape} dtype: {valid_dataset[0][0].dtype}')
    print(f'Validation data attention_mask shape: {valid_dataset[0][1].shape} dtype: {valid_dataset[0][1].dtype}')
    print(f'Validation data targets shape: {valid_dataset[1].shape} dtype: {valid_dataset[1].dtype}')
    
    #Create model
    tf.keras.backend.clear_session()
    model = get_model()
    
    #Compile model with an approximation of layer-wise learning rate decay
    LR_SCH_DECAY_STEPS = 2 * len(train_df) // BATCH_SIZE
    lr_schedule_1 = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-5, 
        decay_steps=LR_SCH_DECAY_STEPS, 
        decay_rate=0.3)
    lr_schedule_2 = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4, 
        decay_steps=LR_SCH_DECAY_STEPS, 
        decay_rate=0.3)
    optimizers = [tf.keras.optimizers.Adam(learning_rate=lr_schedule_1),
                  tf.keras.optimizers.Adam(learning_rate=lr_schedule_2)]
    optimizers_and_layers = [(optimizers[0], model.layers[:-4]),
                             (optimizers[1], model.layers[-4:]),]
    optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

    model.compile(optimizer=optimizer,
                 loss='MeanSquaredError',
                 metrics=[tf.keras.metrics.RootMeanSquaredError()],
                 )
    print('Model prepared.')
    
    #Training model
    print('Start training...')
    callbacks = [
    tf.keras.callbacks.ModelCheckpoint(f"best_model_fold{fold}.h5",
                                       monitor="val_loss",
                                       mode="min",
                                       save_best_only=True,
                                       verbose=1,
                                       save_weights_only=True,),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                     min_delta=1e-5, 
                                     patience=3, 
                                     verbose=1,
                                     mode='min',)
    ]
    history = model.fit(x=train_dataset[0],
                        y=train_dataset[1],
                        validation_data=valid_dataset, 
                        epochs=10,
                        shuffle=True,
                        batch_size=BATCH_SIZE,
                        callbacks=callbacks
                       )
    
    valid_rmses.append(np.min(history.history['val_root_mean_squared_error']))
    print('Training finished.')
    del train_dataset, valid_dataset, train_df, valid_df
    gc.collect()


-----------FOLD 0 ------------
Data prepared.
Training data input_ids shape: (3129, 512) dtype: int32
Training data attention_mask shape: (3129, 512) dtype: int32
Training data targets shape: (3129, 6) dtype: float32
Validation data input_ids shape: (782, 512) dtype: int32
Validation data attention_mask shape: (782, 512) dtype: int32
Validation data targets shape: (782, 6) dtype: float32
Model prepared.
Start training...
Epoch 1/10


2022-10-31 21:31:26.822326: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


783/783 [==============================] - 422s 482ms/step - loss: 0.4319 - root_mean_squared_error: 0.6572 - val_loss: 0.2210 - val_root_mean_squared_error: 0.4701

Epoch 00001: val_loss improved from inf to 0.22103, saving model to best_model_fold0.h5
Epoch 2/10
783/783 [==============================] - 371s 473ms/step - loss: 0.2168 - root_mean_squared_error: 0.4656 - val_loss: 0.2188 - val_root_mean_squared_error: 0.4677

Epoch 00002: val_loss improved from 0.22103 to 0.21876, saving model to best_model_fold0.h5
Epoch 3/10
783/783 [==============================] - 371s 474ms/step - loss: 0.1973 - root_mean_squared_error: 0.4442 - val_loss: 0.2059 - val_root_mean_squared_error: 0.4537

Epoch 00003: val_loss improved from 0.21876 to 0.20588, saving model to best_model_fold0.h5
Epoch 4/10
783/783 [==============================] - 371s 474ms/step - loss: 0.1856 - root_mean_squared_error: 0.4308 - val_loss: 0.2088 - val_root_mean_squared_error: 0.4569

Epoch 00004: val_loss did not i

In [15]:
print(f'{len(valid_rmses)} Folds validation RMSE:\n{valid_rmses}')
print(f'Local CV Average score: {np.mean(valid_rmses)}')

5 Folds validation RMSE:
[0.45372921228408813, 0.46091246604919434, 0.4722799062728882, 0.458873450756073, 0.45331820845603943]
Local CV Average score: 0.45982264876365664


# Inference and Submission

Load test data

In [16]:
test_df = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
test_df.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [17]:
test_dataset = deberta_encode(test_df['full_text'])

# 5 Folds ensemble prediction

In [18]:
fold_preds = []
for fold in range(N_FOLD):
    tf.keras.backend.clear_session()
    model = get_model()
    model.load_weights(f'best_model_fold{fold}.h5')
    print(f'\nFold {fold} inference...')
    pred = model.predict(test_dataset, batch_size=BATCH_SIZE)
    fold_preds.append(pred)
    gc.collect()


Fold 0 inference...

Fold 1 inference...

Fold 2 inference...

Fold 3 inference...

Fold 4 inference...


In [19]:
preds = np.mean(fold_preds, axis=0)
preds = np.clip(preds, 1, 5)

In [20]:
sub_df = pd.concat([test_df[['text_id']], pd.DataFrame(preds, columns=TARGET_COLS)], axis=1)
sub_df.to_csv('submission.csv', index=False)

In [21]:
sub_df.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.958955,2.778274,3.101005,2.931064,2.712532,2.725166
1,000BAD50D026,2.593868,2.408805,2.656760,2.336524,2.128315,2.608301
2,00367BB2546B,3.544802,3.359350,3.521230,3.456447,3.348444,3.287421
